In [249]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from decimal import Decimal 
from datetime import datetime
import math

In [250]:
df = pd.read_csv ('Newfile_RF-04_DISCOVER-AQ', dtype='float', na_values = '-999999')

In [251]:
derivative = df['GPS_ALT'].diff(periods=30) / df['UTC'].diff(periods=30) 
df['trend'] = derivative.gt(0).map({False: -1, True: 1})
t = df['trend'].to_numpy() 

In [252]:
def dataset(trend):
    if trend >= 0:
        
        '''Flight observations while the plane is going up == Difference in altitude for subsequnt points is always poistive.
           So, the difference will be negative whenever the plane starts to come down.
           Divide the dataset according to indices when the plane starts coming down'''
        
        idx = df.index[df['trend'] == 1].dropna()
        dataset_profile = df.loc[idx].dropna(subset=['GPS_ALT', 'THETA'])
        reqd_Index = np.where(dataset_profile['GPS_ALT'].diff() < -1.0)
    else:
        
        '''Flight observations while the plane is coming down == Difference in altitude for subsequnt points is always negative.
           Divide the dataset according to indices when the plane starts going up and the difference becomes positive'''
        
        idx = df.index[df['trend'] == -1].dropna()
        dataset_profile = df.loc[idx].dropna(subset=['GPS_ALT', 'THETA'])
        reqd_Index = np.where(dataset_profile['GPS_ALT'].diff() > 1.0)
    
    final_index = np.asarray(reqd_Index[0])
    diff = np.diff(final_index)
    empty_array = np.arange(0,len(diff),1)
        
    df_list=[]
    i = 0
    for i in empty_array:
        start = final_index[i]
        end = diff[i]+start
        dfx = dataset_profile.iloc[start:end]
        df_list.append(dfx)
    return df_list

Up_sounding = dataset(1)
Down_sounding = dataset(-1)


In [253]:
'''Determine Boundary Layer from Potential Temperature Gradient Method'''

def bl_grad(dataframe):
    alt = dataframe['GPS_ALT']*1000
    theta = dataframe['THETA']
    derivative = ((theta.diff()/alt.diff()).dropna()).to_numpy()
    alt_numpy = alt[1:].to_numpy()
    bl_value = alt_numpy[derivative.argmax()]
    return bl_value

bl_list_ascent =[]
bl_list_descent = []

dataset = 0
for dataset in Up_sounding:
    bl_value = bl_grad(dataset)
    bl_list_ascent.append(bl_value)
    dataset+=1

print(bl_list_ascent,"Units in m")

dataset = 0
for dataset in Down_sounding:
    bl_value = bl_grad(dataset)
    bl_list_descent.append(bl_value)
    dataset+=1

print(bl_list_descent,"Units in m")

[151.59, 1520.12, 4008.2000000000003, 335.96, 4345.98, 359.21999999999997, 348.49, 2754.6499999999996, 603.0799999999999, 351.71000000000004, 1599.48] Units in m
[109.78, 297.48999999999995, 4630.25, 295.9, 4024.22, 568.1, 1566.0, 4639.86, 3781.61, 3986.52, 348.74] Units in m


In [254]:
def consecutive_avg(x,y):
    x = y = pd.DataFrame()
    i = 0
    for i in x:
        for i in y:
            avg = (x[i]+x[i+1])/(y[i+1]-y[i])
    return avg


In [256]:
'''Determination of BL via Bulk Richardson Number technique'''

def bl_richard(dataframe, threshold):
    g = 9.8 #Gravitational Acceleration
    df["WND"] = [math.radians(i) for i in df["WND"]]
    df["WND"] = [math.sin(i) for i in df["WND"]]
    alt = dataframe['GPS_ALT']*1000
    u = dataframe["WNS"]*(dataframe["WND"])
    v = dataframe["WND"]*(dataframe["WND"])
    delta_u = u.diff()
    delta_v = v.diff()
    delta_theta = dataframe["THETA"].diff()
    delta_alt = dataframe["GPS_ALT"].diff()
    avg_theta = dataframe["THETA"].rolling(2).sum()/2
    
    richard_no = (g * delta_alt * delta_theta)/(avg_theta * (delta_u**2 + delta_v**2))
    ind = np.where(richard_no > threshold)
    # print(ind[0][0])
    
    bl_value = alt.to_numpy()[ind[0]]
    return bl_value

bl_richard_ascent = []

for dataset in Up_sounding:
    bl_value = bl_richard(dataset, 0.5)
    bl_richard_ascent.append(bl_value)
    dataset+=1
    
# print(bl_richard_ascent[0])


                

In [100]:
# ''' View al the ascent soundings and pbl values'''
# nrow=4
# ncol=3

# fig, axes = plt.subplots(nrow, ncol, figsize = (8,14), sharex=True, sharey=True)
# plt.suptitle('Potential Temperature Profiles(K): Flight Ascent (2013/9/12)', fontsize = 15)

# count=0
# for r in range(nrow):
#     for c in range(ncol):
#         ax = Up_sounding[count].plot(ax=axes[r,c],y = "GPS_ALT", x = "THETA", label="Theta")
#         ax.axhline(y = bl_list_ascent[count], color ='r')
#         count+=1
# plt.legend()        
# plt.tight_layout()
# plt.show()




In [8]:
'''View the ascent and descent profiles of airplane altitude with respect to time'''
# nrow=4
# ncol=3

# fig, axes = plt.subplots(nrow, ncol, figsize = (8,14), sharey=True)
# plt.suptitle('Flight Profiles', fontsize = 15)

# count=0
# for r in range(nrow):
#     for c in range(ncol):
#         ax1 = Up_sounding[count].plot(ax=axes[r,c],y = "GPS_ALT", x = "UTC", label="Up_Flight Profile")
#         ax2 = ax1.twiny()
#         Down_sounding[count].plot(ax=ax2,y = "GPS_ALT", x = "UTC", color = 'orange', label="Down_Flight Profile")
#         plt.tick_params('x', labelbottom=False)
#         # ax2.set_xticks((0.5, 1.5, 2.5))
#         ax2.set_xlabel("Flight Profile")
        
# plt.legend()        
# plt.tight_layout()
# plt.show()

'View the ascent and descent profiles of airplane altitude with respect to time'

# Final Soundings [Up(Blue) and Red(Down) Profiles]

In [9]:
# nrow=4
# ncol=3

# fig, axes = plt.subplots(nrow, ncol, figsize = (8,12), sharex=True, sharey=True)
# plt.suptitle('Potential Temperature Profiles(K): Reference Flight 4', fontsize = 15)
# axes.axis('off')

# count=0
# for r in range(nrow):
#     for c in range(ncol):
#         ax1 = Up_sounding[count].plot(ax=axes[r,c],y = "GPS_ALT", x = "THETA", label = "UP")
#         ax1.axis('off')
#         ax2 = ax1.twinx()
#         Down_sounding[count].plot(y = "GPS_ALT", x = "THETA", ax=ax2, color = 'red', label = "DOWN")
#         count+=1     

# plt.tight_layout()
# plt.show()